# 解説をつけながら、実行していく。

## まずは、完全ローカルPCで学習ジョブのように実行できること

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!chmod +x opt/ml/input/data/src/*

### プログラム実行 ------------

In [ ]:
!opt/program/train

### ---------------

# 1.コンテナ準備
## 1-1.Dockerfile (確認のみ)

Dockerfileには、構築したいイメージが記述されています。これは、実行したいシステムの完全なオペレーティングシステムのインストールを記述していると考えることができます。しかし、Dockerコンテナの実行は、基本的な操作のためにホストマシン上のLinuxを利用するため、完全なオペレーティングシステムよりもかなり軽量です。

Pythonサイエンススタックでは、標準的なUbuntuのインストールから始めて、通常のツールを実行してscikit-learnで必要なものをインストールします。最後に、特定のアルゴリズムを実装したコードをコンテナに追加して、実行に適した環境を整えます。

その際、余分なスペースを整理します。これにより、コンテナは小さくなり、起動も速くなります。

例のDockerfileを見てみましょう。

imageには、trainやbacktestに必要なソースは含めないこととする。
（学習ジョブ実行時にS3からコピーする）
よって、dockerイメージ作成時に必要な資材はない。


In [ ]:
!cat container/train_v1.0/Dockerfile

## 1-2.Building and registering the container to ECR

以下のシェルコードは、`docker build`を使用してコンテナイメージをビルドし、`docker push`を使用してコンテナイメージをECRにプッシュする方法を示しています。このコードはシェルスクリプト `container/build-and-push.sh` としても提供されており、`build-and-push.sh decision_trees_sample` として実行することで、イメージ `decision_trees_sample` をビルドすることができます。

このコードは、使用しているアカウントと現在のデフォルトリージョン（SageMakerのノートブックインスタンスを使用している場合は、ノートブックインスタンスが作成されたリージョンになります）でECRリポジトリを探します。
リポジトリが存在しない場合、スクリプトはそれを作成します。

https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/container/build_and_push.sh

以下は、build-and-push.shと同じ内容

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=test-rd

cd container/train_v1.0

chmod +x program/train
#chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 疑問：build and push したイメージの動作確認はローカルでできる？
できる。ローカルモード

# -----コンテナ準備完了-----

# 2. データとプログラムをS3にアップロード
・SageMaker　SDKを使う場合。  

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html


・boto3でs3クライアントを使う場合。  

In [ ]:
import sagemaker as sage
sess = sage.Session()

# S3 prefix
#prefix = 'DEMO-scikit-byo-iris'
prefix = 'test-rd/src'

In [ ]:
#WORK_DIRECTORY = 'data'
#data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

src_location = sess.upload_data('opt/ml/input/data/src', bucket='work-aws-virginia', key_prefix=prefix)

# -----データ準備完了-----

# 3-1.学習ジョブ発行(ローカルモード)

In [ ]:
#data_location = 's3://work-aws-virginia/test-rd/candles/'
data_location = 's3://work-aws-virginia/test-rd/train/'


In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

#image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

image = '{}.dkr.ecr.{}.amazonaws.com/test-rd:latest'.format(account, region)

In [ ]:
image

In [ ]:
hyperparameters={'candle_window': 15,
                 'horizon': 300,
                 'target': 'tgt_diff'}

In [ ]:
role = 'arn:aws:iam::805433377179:role/sagemaker-sdk-for-local'

lgbm = sage.estimator.Estimator(image_uri=image,
                                #entasdfdfry_point1='run.sh',
                                #source_sddddddddddir='src',
                                role=role, 
                                instance_count=1,
                                instance_type='local',
                                #instance_type='ml.c4.2xlarge',
                                output_path="s3://{}/output".format(sess.default_bucket()),
                                hyperparameters=hyperparameters,
                                #sagemaker_session=sess)
)

In [ ]:
lgbm.fit({'train':'s3://work-aws-virginia/test-rd/train/',
          'valid':'s3://work-aws-virginia/test-rd/train/',
          'test' :'s3://work-aws-virginia/test-rd/train/',
          'backtest':'s3://work-aws-virginia/test-rd/train/',
          'src':'s3://work-aws-virginia/test-rd/src/'})

# 3-2.学習ジョブ発行(SageMaker)

### 下準備

In [73]:
import sagemaker as sage
sess = sage.Session()

prefix = 'test-rd/src'

In [74]:
### ソースコードをアップロード
src_location = sess.upload_data('opt/ml/input/data/src', bucket='work-aws-virginia', key_prefix=prefix)

### ハイパーパラメータ設定（ローカルファイルから読み込み）
hyperparameters={"timeframe": 15,
                 "horizon" : 300,
                 "hist_row" : 2
}
print(hyperparameters)

{'timeframe': 15, 'horizon': 300, 'hist_row': 2}


In [75]:
HYPERPARAMETER_JSON_PATH = "../../../../../opt/ml/input/config/hyperparameters.json"
HYPERPARAMETER_JSON_PATH = "opt/ml/input/config/hyperparameters.json"

import json
### ハイパーパラメータ設定（ローカルファイルから読み込み）
with open(HYPERPARAMETER_JSON_PATH, "r") as f:
    hyperparameters = json.load(f)

print(hyperparameters)

{'timeframe': 15, 'horizon': 300, 'hist_row': 2, 'target_col': 'tgt_diff'}


In [76]:
role = 'arn:aws:iam::805433377179:role/sagemaker-sdk-for-local'
#role = 'arn:aws:iam::805433377179:role/service-role/AmazonSageMaker-ExecutionRole-20191212T111531'

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/test-rd:latest'.format(account, region)

lgbm = sage.estimator.Estimator(image_uri=image,
                                #entasdfdfry_point1='run.sh',
                                #source_sddddddddddir='src',
                                role=role, 
                                instance_count=1,
                                #instance_type='local',
                                instance_type='ml.c4.2xlarge',
                                #output_path="s3://{}/output".format(sess.default_bucket()),
                                output_path='s3://work-aws-virginia/test-rd/output/',
                                #sagemaker_session=sess # ノートブックインスタンスで実行する場合にIAMを渡す
                                hyperparameters=hyperparameters,
                                )

In [77]:
lgbm.fit({'train':'s3://work-aws-virginia/test-rd/train/',
          'valid':'s3://work-aws-virginia/test-rd/valid/',
          'test' :'s3://work-aws-virginia/test-rd/test/',
          #'backtest':'s3://work-aws-virginia/test-rd/test/',
          'src':'s3://work-aws-virginia/test-rd/src/'},
          wait=False
)



https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

・並行してジョブ発行する場合は、wait=Falseを使う

## 並列実行

In [4]:
import sagemaker as sage
sess = sage.Session()

prefix = 'test-rd/src'
### ソースコードをアップロード
src_location = sess.upload_data('opt/ml/input/data/src', bucket='work-aws-virginia', key_prefix=prefix)

In [5]:
hyperparam_list = [
    'opt/ml/input/config/hyperparameters_10_180_2_diff.json',
    'opt/ml/input/config/hyperparameters_10_300_2_diff.json',
    'opt/ml/input/config/hyperparameters_15_180_2_diff.json',
    'opt/ml/input/config/hyperparameters_15_300_2_diff.json',
    ]

In [6]:
role = 'arn:aws:iam::805433377179:role/sagemaker-sdk-for-local'
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/test-rd:latest'.format(account, region)

import json
for hp_path in hyperparam_list:
    print(hp_path)

    ### ハイパーパラメータ設定（ローカルファイルから読み込み）
    with open(hp_path, "r") as f:
        hyperparameters = json.load(f)

    print(hyperparameters)

    lgbm = sage.estimator.Estimator(image_uri=image,
                                role=role, 
                                instance_count=1,
                                instance_type='ml.c4.2xlarge',
                                output_path='s3://work-aws-virginia/test-rd/output/',
                                hyperparameters=hyperparameters,
                                )
    lgbm.fit({'train':'s3://work-aws-virginia/test-rd/train/',
          'valid':'s3://work-aws-virginia/test-rd/valid/',
          'test' :'s3://work-aws-virginia/test-rd/test/',
          'src':'s3://work-aws-virginia/test-rd/src/'},
          wait=False
          )

opt/ml/input/config/hyperparameters_10_180_2_diff.json
{'timeframe': 10, 'horizon': 180, 'hist_row': 2, 'target_col': 'tgt_diff'}
opt/ml/input/config/hyperparameters_10_300_2_diff.json
{'timeframe': 10, 'horizon': 300, 'hist_row': 2, 'target_col': 'tgt_diff'}
opt/ml/input/config/hyperparameters_15_180_2_diff.json
{'timeframe': 15, 'horizon': 180, 'hist_row': 2, 'target_col': 'tgt_diff'}
opt/ml/input/config/hyperparameters_15_300_2_diff.json
{'timeframe': 15, 'horizon': 300, 'hist_row': 2, 'target_col': 'tgt_diff'}


# 上記のEstimatorを編集していく
https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#

classsagemaker.estimator.Estimator(image_uri, role, instance_count=None, instance_type=None, volume_size=30, volume_kms_key=None, max_run=86400, input_mode='File', output_path=None, output_kms_key=None, base_job_name=None, sagemaker_session=None, hyperparameters=None, tags=None, subnets=None, security_group_ids=None, model_uri=None, model_channel_name='model', metric_definitions=None, encrypt_inter_container_traffic=False, use_spot_instances=False, max_wait=None, checkpoint_s3_uri=None, checkpoint_local_path=None, enable_network_isolation=False, rules=None, debugger_hook_config=None, tensorboard_output_config=None, enable_sagemaker_metrics=None, profiler_config=None, disable_profiler=False, environment=None, **kwargs)

In [ ]:
import sagemaker as sage

In [ ]:
sage.__version__

In [ ]:
!pip install sagemaker==2.33.0

SageMaker Python SDK version == 2.33.0　でOK



SageMaker Python SDKのローカルモードを利用して、ノートブックインスタンス以外の環境で学習ジョブを回してみる
https://dev.classmethod.jp/articles/sagemaker-python-sdk-localmode/

# 完了